### This notebook will only work with GPU support because it calls CUDA functions

In [1]:
!pip install open3d


In [2]:

import open3d as o3d
import numpy as np

print(o3d.__version__)
print(np.__version__)


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
0.18.0
1.26.4


In [4]:

# Tensor from list.
a = o3d.core.Tensor([1, 2, 3])
print("Created from list:\n{}".format(a))

vals = np.array((range(24))).reshape(2, 3, 4)
a = o3d.core.Tensor(vals, dtype=o3d.core.Dtype.Float64, device=o3d.core.Device("CUDA:0"))
print(f"a.shape: {a.shape}")
print(f"a.strides: {a.strides}")
print(f"a.dtype: {a.dtype}")
print(f"a.device: {a.device}")
print(f"a.ndim: {a.ndim}")

# Host -> Device.
a_cpu = o3d.core.Tensor([0, 1, 2])
a_gpu = a_cpu.cuda(0)
print(a_gpu)

# Device -> Host.
a_gpu = o3d.core.Tensor([0, 1, 2], device=o3d.core.Device("CUDA:0"))
a_cpu = a_gpu.cpu()
print(a_cpu)

# Device -> another Device.
a_gpu_0 = o3d.core.Tensor([0, 1, 2], device=o3d.core.Device("CUDA:0"))
a_gpu_1 = a_gpu_0.cuda(0)
print(a_gpu_1)


Created from list:
[1 2 3]
Tensor[shape={3}, stride={1}, Int64, CPU:0, 0x57758f2cacf0]


RuntimeError: [1;31m[Open3D Error] (static std::shared_ptr<open3d::core::MemoryManagerDevice> open3d::core::MemoryManager::GetMemoryManagerDevice(const open3d::core::Device&)) /root/Open3D/cpp/open3d/core/MemoryManager.cpp:123: Unsupported device "CUDA:0". Set BUILD_CUDA_MODULE=ON to compile for CUDA support and BUILD_SYCL_MODULE=ON to compile for SYCL support.
[0;m

In [ ]:

import torch
import torch.utils.dlpack

# From PyTorch
th_a = torch.ones((5,)).cuda(0)
o3_a = o3d.core.Tensor.from_dlpack(torch.utils.dlpack.to_dlpack(th_a))
print(f"th_a: {th_a}")
print(f"o3_a: {o3_a}")
print("")

# Changes to PyTorch array reflects on open3d Tensor and vice versa
th_a[0] = 100
o3_a[1] = 200
print(f"th_a: {th_a}")
print(f"o3_a: {o3_a}")


demo_icp_pcds = o3d.data.DemoICPPointClouds()


In [ ]:


import copy
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp],
                                      zoom=0.4459,
                                      front=[0.9288, -0.2951, -0.2242],
                                      lookat=[1.6784, 2.0612, 1.4451],
                                      up=[-0.3402, -0.9189, -0.1996])

source = o3d.io.read_point_cloud(demo_icp_pcds.paths[0])
target = o3d.io.read_point_cloud(demo_icp_pcds.paths[1])
threshold = 0.02
trans_init = np.asarray([[0.862, 0.011, -0.507, 0.5],
                         [-0.139, 0.967, -0.215, 0.7],
                         [0.487, 0.255, 0.835, -1.4], [0.0, 0.0, 0.0, 1.0]])


# draw_registration_result(source, target, trans_init)


In [ ]:

#### Visualize with plotly

https://medium.com/@sim30217/visualize-point-cloud-using-open3d-in-colab-a05b2f780a96

import plotly.graph_objects as go

print("Load a ply point cloud, print it, and render it")
ply_point_cloud = o3d.data.PLYPointCloud()
points = np.asarray(source.points)
# print(source)
# print(np.asarray(source.points))

colors = None
if source.has_colors():
    colors = np.asarray(source.colors)
elif source.has_normals():
    colors = (0.5, 0.5, 0.5) + np.asarray(source.normals) * 0.5


fig = go.Figure(
  data=[
    go.Scatter3d(
      x=points[:,0], y=points[:,1], z=points[:,2],
      mode='markers',
      marker=dict(size=1, color=colors)
)
],
  layout=dict(
    scene=dict(
      xaxis=dict(visible=False),
      yaxis=dict(visible=False),
      zaxis=dict(visible=False)
)
)
)
fig.show()

* Google colab examples - https://colab.research.google.com/drive/1CR_HDvJ2AnjJV3Bf5vwP70K0hx3RcdMb?usp=sharing#scrollTo=WG5EhO6R_5XD

In [ ]:
def draw_geometries(geometries):
    graph_objects = []

    for geometry in geometries:
        geometry_type = geometry.get_geometry_type()
        
        if geometry_type == o3d.geometry.Geometry.Type.PointCloud:
            points = np.asarray(geometry.points)
            colors = None
            if geometry.has_colors():
                colors = np.asarray(geometry.colors)
            elif geometry.has_normals():
                colors = (0.5, 0.5, 0.5) + np.asarray(geometry.normals) * 0.5
            else:
                geometry.paint_uniform_color((1.0, 0.0, 0.0))
                colors = np.asarray(geometry.colors)

            scatter_3d = go.Scatter3d(x=points[:,0], y=points[:,1], z=points[:,2], mode='markers', marker=dict(size=1, color=colors))
            graph_objects.append(scatter_3d)

        if geometry_type == o3d.geometry.Geometry.Type.TriangleMesh:
            triangles = np.asarray(geometry.triangles)
            vertices = np.asarray(geometry.vertices)
            colors = None
            if geometry.has_triangle_normals():
                colors = (0.5, 0.5, 0.5) + np.asarray(geometry.triangle_normals) * 0.5
                colors = tuple(map(tuple, colors))
            else:
                colors = (1.0, 0.0, 0.0)
            
            mesh_3d = go.Mesh3d(x=vertices[:,0], y=vertices[:,1], z=vertices[:,2], i=triangles[:,0], j=triangles[:,1], k=triangles[:,2], facecolor=colors, opacity=0.50)
            graph_objects.append(mesh_3d)
        
    fig = go.Figure(
        data=graph_objects,
        layout=dict(
            scene=dict(
                xaxis=dict(visible=False),
                yaxis=dict(visible=False),
                zaxis=dict(visible=False)
            )
        )
    )
    fig.show()

In [ ]:
o3d.visualization.draw_geometries = draw_geometries # replace function
o3d.visualization.draw_geometries([source])
# o3d.visualization.draw_geometries([mesh])